## Data enrichment with Google Places API

In [48]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import folium

In [29]:
df_1500 = pd.read_csv("for_api_1500.csv")
display(df.head(12))
print(df.shape)

,Unnamed: 0,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices,nearest_offices,ranking
0,210,5d277baf5370f8e6fc94d96c,games_video,USA,2008,"{'type': 'Point', 'coordinates': [-71.0680027,...",42.340653,-71.068003,SCVNGR,60,$40.8M,1,27,13.5
1,53,5d277baf5370f8e6fc94d8cf,advertising,USA,2006,"{'type': 'Point', 'coordinates': [-122.245087,...",37.530893,-122.245087,MyBuys,120,$40.8M,2,27,13.5
2,189,5d277baf5370f8e6fc94d957,software,USA,2003,"{'type': 'Point', 'coordinates': [-121.970505,...",37.382137,-121.970505,Packet Design,60,$2.8M,1,27,13.5
3,125,5d277baf5370f8e6fc94d917,advertising,USA,2008,"{'type': 'Point', 'coordinates': [-122.4011706...",37.791119,-122.401171,Bizo,66,$28.5M,1,27,13.5
4,78,5d277baf5370f8e6fc94d8e8,advertising,USA,2004,"{'type': 'Point', 'coordinates': [-73.984268, ...",40.743662,-73.984268,MediaTrust,60,$21M,1,27,13.5
5,206,5d277baf5370f8e6fc94d968,software,GBR,2005,"{'type': 'Point', 'coordinates': [-1.4629301, ...",53.380011,-1.462930,WANdisco,175,$45M,2,27,13.5
6,20,5d277baf5370f8e6fc94d8ae,web,USA,2006,"{'type': 'Point', 'coordinates': [-122.39071, ...",37.779507,-122.390710,Eventbrite,200,$140M,1,27,13.5
7,24,5d277baf5370f8e6fc94d8b2,web,USA,2004,"{'type': 'Point', 'coordinates': [-122.0727749...",37.392798,-122.072775,IMVU,150,$29M,1,27,13.5
8,88,5d277baf5370f8e6fc94d8f2,design,DEU,2008,"{'type': 'Point', 'coordinates': [13.4469031, ...",52.498620,13.446903,99designs,100,$35M,3,27,13.5
9,196,5d277baf5370f8e6fc94d95e,web,USA,2004,"{'type': 'Point', 'coordinates': [-122.3927145...",37.788714,-122.392714,Double Fusion,80,$59M,1,27,13.5


(239, 14)


In [30]:
# We apply filter to reduce number of companies.
# As we consider the most importat the number of near companies, employees and and year, we filter for its first 10

In [31]:
df_1500_api = df_1500.head(10)

In [32]:
df_1500_api.reset_index(inplace = True)

In [33]:
"""taxi_stand
train_station
transit_station
subway_station
bus_station
"""
def ranking_places(col):
    rank_scores = {"nearest_offices": 0.8, "total_money_raised": 0.7, "number_of_employees": 0.7, "meetups": 0.6, 
                   "nursery": 0.3, "gym": 0.3}
    match = []
    for i in range(len(col)):
        match.append(rank_scores.get(col[i]) * rank_scores.value[i])
    return match

In [34]:
df_1500_api.head(15)

,level_0,Unnamed: 0,index,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices,nearest_offices
0,0,0,51,5d277baf5370f8e6fc94d8cd,enterprise,USA,2006,"{'type': 'Point', 'coordinates': [-122.158804,...",37.439380,-122.158804,Gigya,230,$69.8M,1,27
1,1,1,20,5d277baf5370f8e6fc94d8ae,web,USA,2006,"{'type': 'Point', 'coordinates': [-122.39071, ...",37.779507,-122.390710,Eventbrite,200,$140M,1,27
2,2,2,167,5d277baf5370f8e6fc94d941,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-121.967223,...",37.369445,-121.967223,Apigee,65,$107M,1,27
3,3,3,26,5d277baf5370f8e6fc94d8b4,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-104.988123,...",39.755092,-104.988123,NewsGator,90,$39.3M,1,27
4,4,4,190,5d277baf5370f8e6fc94d958,advertising,USA,2006,"{'type': 'Point', 'coordinates': [-105.278785,...",40.058448,-105.278785,Intela,70,$13.3M,4,27
5,5,5,136,5d277baf5370f8e6fc94d922,advertising,SWE,2007,"{'type': 'Point', 'coordinates': [18.0333601, ...",59.335183,18.033360,Videoplaza,70,€15.9M,5,27
6,6,6,165,5d277baf5370f8e6fc94d93f,software,USA,2005,"{'type': 'Point', 'coordinates': [-122.161287,...",37.444098,-122.161287,Nexenta Systems,200,$49.9M,1,27
7,7,7,85,5d277baf5370f8e6fc94d8ef,web,USA,2006,"{'type': 'Point', 'coordinates': [-73.7258199,...",40.858752,-73.725820,OLX,85,$28.5M,1,27
8,8,8,164,5d277baf5370f8e6fc94d93e,web,USA,2006,"{'type': 'Point', 'coordinates': [-111.643179,...",40.250096,-111.643179,FamilyLink,60,$3.65M,1,27
9,9,9,27,5d277baf5370f8e6fc94d8b5,advertising,USA,2005,"{'type': 'Point', 'coordinates': [-122.39323, ...",37.710677,-122.393230,Greystripe,90,$17.6M,1,27


In [40]:
PLACES_KEY = os.environ["PLACES_KEY"]


output_file = "json"
radius = "1500"
type_= "restaurant"
keywords = "hamburguers"
lst_restau = []

for i in range(len(df_1500_api)):
    coor = df_1500_api["latitude"][i].astype(str) + ", " + df_1500_api["longitude"][i].astype(str)
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/"+ output_file +"?location="+coor +"&radius=" +radius+ "&type="+type_+"&keyword="+keywords + "&key="+ PLACES_KEY
    res = requests.get(url)
    data = res.json()
    lst_restau.append(len(data))

In [42]:
print(lst_restau)
len(lst_restau)
df_1500_api["num_restaurants"] = lst_restau

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


/home/slimbook/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
df_1500_api.head(10)

,level_0,Unnamed: 0,index,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices,nearest_offices,num_restaurants
0,0,0,51,5d277baf5370f8e6fc94d8cd,enterprise,USA,2006,"{'type': 'Point', 'coordinates': [-122.158804,...",37.439380,-122.158804,Gigya,230,$69.8M,1,27,3
1,1,1,20,5d277baf5370f8e6fc94d8ae,web,USA,2006,"{'type': 'Point', 'coordinates': [-122.39071, ...",37.779507,-122.390710,Eventbrite,200,$140M,1,27,3
2,2,2,167,5d277baf5370f8e6fc94d941,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-121.967223,...",37.369445,-121.967223,Apigee,65,$107M,1,27,3
3,3,3,26,5d277baf5370f8e6fc94d8b4,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-104.988123,...",39.755092,-104.988123,NewsGator,90,$39.3M,1,27,3
4,4,4,190,5d277baf5370f8e6fc94d958,advertising,USA,2006,"{'type': 'Point', 'coordinates': [-105.278785,...",40.058448,-105.278785,Intela,70,$13.3M,4,27,3
5,5,5,136,5d277baf5370f8e6fc94d922,advertising,SWE,2007,"{'type': 'Point', 'coordinates': [18.0333601, ...",59.335183,18.033360,Videoplaza,70,€15.9M,5,27,3
6,6,6,165,5d277baf5370f8e6fc94d93f,software,USA,2005,"{'type': 'Point', 'coordinates': [-122.161287,...",37.444098,-122.161287,Nexenta Systems,200,$49.9M,1,27,3
7,7,7,85,5d277baf5370f8e6fc94d8ef,web,USA,2006,"{'type': 'Point', 'coordinates': [-73.7258199,...",40.858752,-73.725820,OLX,85,$28.5M,1,27,3
8,8,8,164,5d277baf5370f8e6fc94d93e,web,USA,2006,"{'type': 'Point', 'coordinates': [-111.643179,...",40.250096,-111.643179,FamilyLink,60,$3.65M,1,27,3
9,9,9,27,5d277baf5370f8e6fc94d8b5,advertising,USA,2005,"{'type': 'Point', 'coordinates': [-122.39323, ...",37.710677,-122.393230,Greystripe,90,$17.6M,1,27,3


In [49]:
def points(col_lat, col_lon): #insert values as (df[col_lat], df[col_lon])
    statmad = df_1500_api[['latitude', 'longitude']]
    statmadlist = statmad.values.tolist()
    popup = df_1500_api[['name']]
    statmap = folium.Map(zoom_start=11) 
    for point in range(0, len(statmadlist)):
        folium.Marker(statmadlist[point], popup=popup.iloc[point,0]).add_to(statmap)
    return statmap    

points(df_1500_api['latitude'], df_1500_api['longitude'])

In [52]:
PLACES_KEY = os.environ["PLACES_KEY"]


output_file = "json"
radius = "1500"
type_= "transit_station"
keywords = "bus"
lst_buses = []

for i in range(len(df_1500_api)):
    coor = df_1500_api["latitude"][i].astype(str) + ", " + df_1500_api["longitude"][i].astype(str)
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/"+ output_file +"?location="+coor +"&radius=" +radius+ "&type="+type_+"&keyword="+keywords + "&key="+ PLACES_KEY
    res = requests.get(url)
    data = res.json()
    lst_buses.append(len(data))
    
print(lst_buses)
len(lst_buses)
df_1500_api["buses_stations"] = lst_buses    

[3, 4, 4, 4, 4, 4, 3, 3, 4, 4]


/home/slimbook/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [54]:
PLACES_KEY = os.environ["PLACES_KEY"]


output_file = "json"
radius = "1500"
type_= "car_rental"
keywords = "car+rental"
lst_car = []

for i in range(len(df_1500_api)):
    coor = df_1500_api["latitude"][i].astype(str) + ", " + df_1500_api["longitude"][i].astype(str)
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/"+ output_file +"?location="+coor +"&radius=" +radius+ "&type="+type_+"&keyword="+keywords + "&key="+ PLACES_KEY
    res = requests.get(url)
    data = res.json()
    lst_car.append(len(data))
    
print(lst_car)
len(lst_car)
df_1500_api["car_rental"] = lst_car    

[3, 4, 3, 4, 3, 4, 3, 3, 3, 3]


/home/slimbook/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
PLACES_KEY = os.environ["PLACES_KEY"]


output_file = "json"
radius = "1500"
type_= "gym"
keywords = "gym"
lst_gym = []

for i in range(len(df_1500_api)):
    coor = df_1500_api["latitude"][i].astype(str) + ", " + df_1500_api["longitude"][i].astype(str)
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/"+ output_file +"?location="+coor +"&radius=" +radius+ "&type="+type_+"&keyword="+keywords + "&key="+ PLACES_KEY
    res = requests.get(url)
    data = res.json()
    lst_gym.append(len(data))
    
print(lst_gym)
len(lst_gym)
df_1500_api["num_gym"] = lst_gym    

[3, 4, 3, 4, 3, 4, 3, 3, 3, 3]


/home/slimbook/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
df_1500_api.columns


Index(['level_0', 'Unnamed: 0', 'index', '_id', 'category_code',
       'country_code', 'founded_year', 'geoloc', 'latitude', 'longitude',
       'name', 'number_of_employees', 'total_money_raised', 'total_offices',
       'nearest_offices', 'num_restaurants', 'buses_stations', 'car_rental',
       'num_gym'],
      dtype='object')

In [61]:
df_1500_api["ranking"] = df_1500_api['number_of_employees']*0.8 + df_1500_api['nearest_offices']*0.8 + df_1500_api["num_restaurants"]*0.6 + df_1500_api["car_rental"]*0.5 + df_1500_api["buses_stations"]*0.3 + df_1500_api["num_gym"]*0.3 
df_1500_api.head()

/home/slimbook/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,level_0,Unnamed: 0,index,_id,category_code,country_code,founded_year,geoloc,latitude,longitude,name,number_of_employees,total_money_raised,total_offices,nearest_offices,num_restaurants,buses_stations,car_rental,num_gym,ranking
0,0,0,51,5d277baf5370f8e6fc94d8cd,enterprise,USA,2006,"{'type': 'Point', 'coordinates': [-122.158804,...",37.439380,-122.158804,Gigya,230,$69.8M,1,27,3,3,3,3,210.7
1,1,1,20,5d277baf5370f8e6fc94d8ae,web,USA,2006,"{'type': 'Point', 'coordinates': [-122.39071, ...",37.779507,-122.390710,Eventbrite,200,$140M,1,27,3,4,4,4,187.8
2,2,2,167,5d277baf5370f8e6fc94d941,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-121.967223,...",37.369445,-121.967223,Apigee,65,$107M,1,27,3,4,3,3,79.0
3,3,3,26,5d277baf5370f8e6fc94d8b4,enterprise,USA,2004,"{'type': 'Point', 'coordinates': [-104.988123,...",39.755092,-104.988123,NewsGator,90,$39.3M,1,27,3,4,4,4,99.8
4,4,4,190,5d277baf5370f8e6fc94d958,advertising,USA,2006,"{'type': 'Point', 'coordinates': [-105.278785,...",40.058448,-105.278785,Intela,70,$13.3M,4,27,3,4,3,3,83.0
